# Clear Creek Forecast

Use the GeoJSON Clear Creek dataset to generate a forecast.

In [1]:
from NEXT import NEXT
from NEXT.data import all_data_gpkg, full_data
import matplotlib.pyplot as plt
import geopandas as gp
import pandas as pd
import warnings
import py3dep as p3d

ERROR 1: PROJ: proj_create_from_database: Open of /u/wy/ch/dphilippus/.conda/envs/next/share/proj failed


In [2]:
gpkg_path = "clear_ck.geojson"

In [3]:
geo = gp.read_file(gpkg_path)

In [6]:
geo

,divide_id,toid,type,ds_id,areasqkm,id,lengthkm,tot_drainage_areasqkm,has_flowline,geometry
0,cat-1561662,nex-1561660,network,None,10.272150,wb-1561662,3.912857,17.395650,True,"POLYGON ((-105.38 39.689, -105.39 39.688, -105..."
1,cat-1561693,nex-1561694,network,None,23.875199,wb-1561693,6.565050,29.484450,True,"POLYGON ((-105.74 39.597, -105.74 39.597, -105..."
2,cat-1561753,nex-1561636,network,None,13.279499,wb-1561753,6.903318,27.470700,True,"POLYGON ((-105.34 39.788, -105.34 39.79, -105...."
3,cat-1561618,nex-1561619,network,None,5.856751,wb-1561618,2.987772,40.154401,True,"POLYGON ((-105.85 39.686, -105.86 39.685, -105..."
4,cat-1561619,nex-1561620,network,None,8.104951,wb-1561619,3.502942,62.124751,True,"POLYGON ((-105.81 39.705, -105.81 39.702, -105..."
...,...,...,...,...,...,...,...,...,...,...
114,cat-1561668,nex-1561631,network,None,16.689150,wb-1561668,7.782792,36.214198,True,"POLYGON ((-105.52 39.723, -105.52 39.722, -105..."
115,cat-1561692,nex-1561693,network,None,5.609250,wb-1561692,2.116735,5.609250,True,"POLYGON ((-105.75 39.596, -105.75 39.596, -105..."
116,cat-1561695,nex-1561696,network,None,7.276050,wb-1561695,4.120794,41.594850,True,"POLYGON ((-105.71 39.678, -105.71 39.679, -105..."
117,cat-1561744,nex-1561737,network,None,11.991150,wb-1561744,6.066944,11.991150,True,"POLYGON ((-105.43 39.806, -105.43 39.806, -105..."


In [14]:
data_gen = all_data_gpkg(gpkg_path, "2022-01-01", "2025-03-08", weather="hrrr", handler=lambda k, g, e: print(e))

In [ ]:
indata = pd.concat([df for df in data_gen])

'date'
'date'
'date'
'date'
'date'
